In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
print("API key loaded?", api_key is not None)

API key loaded? True


In [20]:
import sys, os
sys.path.append(os.path.abspath("../src"))

import + sample CSV

In [14]:
import pandas as pd
import json
from pathlib import Path

df = pd.read_csv("../data/sample_sales.csv") 

df.head()

,order_id,customer,category,price,cost,profit,date
0,1001,Alice,Electronics,120.5,85.0,35.5,2024-01-01
1,1002,Bob,Clothing,45.0,20.0,25.0,2024-01-02
2,1003,Cindy,Electronics,89.9,70.0,19.9,2024-01-03
3,1004,David,Furniture,220.0,160.0,60.0,2024-01-04
4,1005,Emma,Clothing,130.0,55.0,75.0,2024-01-05


infer column metadata

In [15]:
def infer_dtype(series):
    if pd.api.types.is_integer_dtype(series):
        return "int"
    if pd.api.types.is_float_dtype(series):
        return "float"
    if pd.api.types.is_bool_dtype(series):
        return "bool"
    if pd.api.types.is_datetime64_any_dtype(series):
        return "datetime"
    return "string"

columns_meta = []

for col in df.columns:
    s = df[col]
    col_meta = {
        "name": col,
        "dtype": infer_dtype(s),
        "nullable": bool(s.isna().any()),
        "example_values": s.dropna().astype(str).head(3).tolist(),
    }

    # 数值列 summary
    if col_meta["dtype"] in ["int", "float"]:
        col_meta["summary"] = {
            "min": float(s.min()),
            "max": float(s.max()),
            "mean": float(s.mean()),
        }

    columns_meta.append(col_meta)

columns_meta

[{'name': 'order_id',
  'dtype': 'int',
  'nullable': False,
  'example_values': ['1001', '1002', '1003'],
  'summary': {'min': 1001.0, 'max': 1007.0, 'mean': 1004.0}},
 {'name': 'customer',
  'dtype': 'string',
  'nullable': False,
  'example_values': ['Alice', 'Bob', 'Cindy']},
 {'name': 'category',
  'dtype': 'string',
  'nullable': False,
  'example_values': ['Electronics', 'Clothing', 'Electronics']},
 {'name': 'price',
  'dtype': 'float',
  'nullable': False,
  'example_values': ['120.5', '45.0', '89.9'],
  'summary': {'min': 45.0, 'max': 950.0, 'mean': 240.05714285714288}},
 {'name': 'cost',
  'dtype': 'float',
  'nullable': False,
  'example_values': ['85.0', '20.0', '70.0'],
  'summary': {'min': 20.0, 'max': 780.0, 'mean': 172.85714285714286}},
 {'name': 'profit',
  'dtype': 'float',
  'nullable': False,
  'example_values': ['35.5', '25.0', '19.9'],
  'summary': {'min': 19.9, 'max': 170.0, 'mean': 67.2}},
 {'name': 'date',
  'dtype': 'string',
  'nullable': False,
  'example_v

build the draft JSON

In [16]:
dataset_json = {
    "dataset_id": "test-001",
    "name": "sample_sales",
    "source_type": "csv_upload",
    "meta": {
        "row_count": int(len(df)),
        "column_count": int(len(df.columns)),
        "columns": columns_meta
    },
    "data_sample": df.head(5).to_dict(orient="records")
}

json.dumps(dataset_json, indent=2)

'{\n  "dataset_id": "test-001",\n  "name": "sample_sales",\n  "source_type": "csv_upload",\n  "meta": {\n    "row_count": 7,\n    "column_count": 7,\n    "columns": [\n      {\n        "name": "order_id",\n        "dtype": "int",\n        "nullable": false,\n        "example_values": [\n          "1001",\n          "1002",\n          "1003"\n        ],\n        "summary": {\n          "min": 1001.0,\n          "max": 1007.0,\n          "mean": 1004.0\n        }\n      },\n      {\n        "name": "customer",\n        "dtype": "string",\n        "nullable": false,\n        "example_values": [\n          "Alice",\n          "Bob",\n          "Cindy"\n        ]\n      },\n      {\n        "name": "category",\n        "dtype": "string",\n        "nullable": false,\n        "example_values": [\n          "Electronics",\n          "Clothing",\n          "Electronics"\n        ]\n      },\n      {\n        "name": "price",\n        "dtype": "float",\n        "nullable": false,\n        "examp

In [17]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

prompt = f"""
You are a data analysis assistant.
Here is the dataset metadata:

{json.dumps(dataset_json['meta'], indent=2)}

Generate 5 meaningful questions a user can explore with this data.
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Here are five meaningful questions that a user can explore with the provided dataset:

1. **Which customer generated the highest profit?**
   - Analyze the profit values associated with each customer to identify who has the highest overall profit contribution.

2. **What is the average profit margin across different product categories?**
   - Calculate the profit margin (profit as a percentage of price) for each category and find the average to determine which categories are more profitable.

3. **How do the prices of products vary by category?**
   - Compare the average prices of products in different categories to see if certain categories tend to have higher or lower price points.

4. **What is the trend of orders over the dates provided?**
   - Examine the number of orders placed each day to identify any trends or patterns within the order dates, such as peak buying days.

5. **What is the relationship between price and profit?**
   - Investigate whether there is a correlation betw

In [1]:
from pathlib import Path
import sys, os
sys.path.append(os.path.abspath("../src"))

from datapipe.json_schema import csv_bytes_to_llm_json

csv_path = Path("../data/sample_sales.csv")
file_bytes = csv_path.read_bytes()

dataset_json = csv_bytes_to_llm_json(file_bytes, dataset_name="sample_sales")
dataset_json["meta"]

{'row_count': 7,
 'column_count': 7,
 'columns': [{'name': 'order_id',
   'dtype': 'int',
   'nullable': False,
   'example_values': ['1001', '1002', '1003'],
   'summary': {'min': 1001.0, 'max': 1007.0, 'mean': 1004.0}},
  {'name': 'customer',
   'dtype': 'string',
   'nullable': False,
   'example_values': ['Alice', 'Bob', 'Cindy']},
  {'name': 'category',
   'dtype': 'string',
   'nullable': False,
   'example_values': ['Electronics', 'Clothing', 'Electronics']},
  {'name': 'price',
   'dtype': 'float',
   'nullable': False,
   'example_values': ['120.5', '45.0', '89.9'],
   'summary': {'min': 45.0, 'max': 950.0, 'mean': 240.05714285714288}},
  {'name': 'cost',
   'dtype': 'float',
   'nullable': False,
   'example_values': ['85.0', '20.0', '70.0'],
   'summary': {'min': 20.0, 'max': 780.0, 'mean': 172.85714285714286}},
  {'name': 'profit',
   'dtype': 'float',
   'nullable': False,
   'example_values': ['35.5', '25.0', '19.9'],
   'summary': {'min': 19.9, 'max': 170.0, 'mean': 67.2